In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Model define - cifar10

In [2]:
from tensorflow.keras.datasets.cifar10 import load_data
(x_train, y_train),(x_test, y_test) = load_data()

In [3]:
print(x_train.shape)

(50000, 32, 32, 3)


In [4]:
y_train.shape

(50000, 1)

In [5]:
class Model:
    def __init__(self,sess,name, learning_rate=0.01):
        self.sess = sess
        self.name = name
        self.learning_rate = learning_rate
        self.training=False
        self.build_model()
        
    def build_model(self):
        nb_classes = 10
        # placeholder
        with tf.variable_scope(self.name):
            self.x = tf.placeholder(tf.float32, [None, 32,32,3])
            self.y = tf.placeholder(tf.int32, [None,1])
            y_one_hot = tf.one_hot(self.y, nb_classes)
            y_one_hot = tf.reshape(y_one_hot, [-1, nb_classes])
            # layer1  input => -1, 32, 32, 3
            l1 = tf.layers.conv2d(inputs=self.x, filters=13, strides=[1,1], kernel_size=[4,4], activation=tf.nn.relu,padding="SAME",
                kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer())
            l1 = tf.layers.max_pooling2d(inputs=l1, pool_size=[2,2], strides=2, padding="SAME")
            l1 = tf.layers.dropout(inputs=l1, rate=0.7, training = self.training)
            # layer2  input => -1, 16, 16, 32
            l2 = tf.layers.conv2d(inputs=l1, filters = 12, strides=[1,1], kernel_size=[4,4], activation=tf.nn.relu,padding="SAME",
                kernel_initializer=tf.contrib.layers.xavier_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer())
            l2 = tf.layers.max_pooling2d(l2, pool_size=[2,2], strides=2, padding="SAME")
            l2 = tf.layers.dropout(l2, rate=0.7, training = self.training)
            # layer3 input => -1, 8, 8, 64
            w3 = tf.get_variable("w3", [4,4,12,11], initializer = tf.contrib.layers.xavier_initializer())
            b3 = tf.get_variable("b3", [11], initializer = tf.contrib.layers.xavier_initializer())
            l3 = tf.nn.conv2d(l2, w3, strides=[1,1,1,1], padding="SAME")+b3
            l3 = tf.nn.max_pool(l3, ksize=[1,2,2,1],strides=[1,2,2,1], padding="SAME")
            l3 = tf.nn.relu(l3)
            l3 = tf.layers.dropout(l3, rate=0.7, training=self.training)
    
            l3 = tf.reshape(l3, [-1,11*4*4])
            # layer4 input =>-1, 4, 4, 128 
            l4 = tf.layers.dense(inputs=l3, units = 20, activation = tf.nn.relu, kernel_initializer = tf.contrib.layers.xavier_initializer(), bias_initializer = tf.contrib.layers.xavier_initializer())
            l4 = tf.layers.dropout(l4, rate=0.7, training=self.training)
            # layer5 input =>
            w5 = tf.get_variable("w5", [20 , nb_classes], initializer = tf.contrib.layers.xavier_initializer())
            b5 = tf.get_variable("b5", [nb_classes], initializer = tf.contrib.layers.xavier_initializer())
            self.logits = tf.matmul(l4, w5) + b5
            # model
            self.hypothesis = tf.nn.softmax(self.logits)
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels = y_one_hot))
            self. optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
            # predicted & accuracy
            predicted = tf.argmax(self.hypothesis, 1)
            self.accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, tf.argmax(y_one_hot,1)), dtype=tf.float32))
        
    def predict(self, x_test, *y_test):
        return self.sess.run(self.hypothesis, feed_dict = {self.x:x_test})
    
    def get_accuracy(self, x_test, y_test):
        return self.sess.run(self.accuracy, feed_dict={self.x:x_test, self.y:y_test})
    
    def train(self, x_train, y_train):
        self.training = True
        return self.sess.run([self.cost,self.optimizer], feed_dict = {self.x:x_train, self.y:y_train})
    

In [6]:
sess=tf.Session()
models = [Model(sess,"modellllll"+str(i)) for i in range(3)]

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# train

In [7]:
# parameters
training_epochs = 15
batch_size = 100
num = len(x_train)//batch_size
# data
#dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
# action
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = np.zeros(len(models))
    
    for step in range(num):
        start_idx = batch_size*step
        end_idx = batch_size*(step+1)
        batch_x = x_train[start_idx:end_idx,:,:,:]
        batch_y = y_train[start_idx:end_idx,:]
        for m_idx, m in enumerate(models):
            c ,_ = m.train(batch_x, batch_y)
            avg_cost[m_idx] += c/num
    print(f"Epoch {epoch} :", avg_cost)

Epoch 0 : [2.46493936 2.55985636 2.62551793]
Epoch 1 : [1.84427803 2.30266379 2.30342132]
Epoch 2 : [1.78832807 2.30408348 2.30356842]
Epoch 3 : [1.75650998 2.30254214 2.30348953]
Epoch 4 : [1.740488   2.26245362 2.30350035]
Epoch 5 : [1.71224739 2.00529513 2.30350634]
Epoch 6 : [1.71589023 1.90984778 2.30350978]
Epoch 7 : [1.70276243 1.85959498 2.30351179]
Epoch 8 : [1.69473453 1.85248178 2.30351299]
Epoch 9 : [1.67966361 1.8292841  2.30351371]
Epoch 10 : [1.68568663 1.81529908 2.30351414]
Epoch 11 : [1.72126347 1.81379611 2.30351439]
Epoch 12 : [1.70978827 1.7959508  2.30351454]
Epoch 13 : [2.02905179 1.827196   2.30351465]
Epoch 14 : [1.95862627 1.77303479 2.30351469]


# test

In [12]:
ensemble_result = np.zeros(len(x_test)*10).reshape(-1,10)

for m_idx, m in enumerate(models):
    print(f"model {m_idx} accuracy :", m.get_accuracy(x_test,y_test))
    p = m.predict(x_test,y_test)
    ensemble_result+=p
    
ensemble_predict = tf.argmax(ensemble_result,1)
ensemble_accuracy = tf.reduce_mean(tf.cast(tf.equal(ensemble_predict,tf.argmax(y_test,1)), dtype=tf.float32))
print(f"ensemble_accuracy : {sess.run(ensemble_accuracy)}")

model 0 accuracy : 0.3523
model 1 accuracy : 0.3583
model 2 accuracy : 0.1
ensemble_accuracy : 0.10189999639987946
